by: Lauren Broussard

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

## Project Assignment:

"Defining  an  "adopted  user"   as  a  user  who   has  logged  into  the  product  on  three  separate days  in  at  least  one  seven­day  period ,  identify  which  factors  predict  future  user adoption"

### LOAD & CLEAN DATA

In [2]:
# load users table
users = pd.read_csv('takehome_users.csv',parse_dates=['creation_time'], encoding="ISO-8859-1")
users.head()

,object_id,creation_time,name,email,creation_source,last_session_creation_time,opted_in_to_mailing_list,enabled_for_marketing_drip,org_id,invited_by_user_id
0,1,2014-04-22 03:53:30,Clausen August,AugustCClausen@yahoo.com,GUEST_INVITE,1.398139e+09,1,0,11,10803.0
1,2,2013-11-15 03:45:04,Poole Matthew,MatthewPoole@gustr.com,ORG_INVITE,1.396238e+09,0,0,1,316.0
2,3,2013-03-19 23:14:52,Bottrill Mitchell,MitchellBottrill@gustr.com,ORG_INVITE,1.363735e+09,0,0,94,1525.0
3,4,2013-05-21 08:09:28,Clausen Nicklas,NicklasSClausen@yahoo.com,GUEST_INVITE,1.369210e+09,0,0,1,5151.0
4,5,2013-01-17 10:14:20,Raw Grace,GraceRaw@yahoo.com,GUEST_INVITE,1.358850e+09,0,0,193,5240.0


In [3]:
users.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12000 entries, 0 to 11999
Data columns (total 10 columns):
 #   Column                      Non-Null Count  Dtype         
---  ------                      --------------  -----         
 0   object_id                   12000 non-null  int64         
 1   creation_time               12000 non-null  datetime64[ns]
 2   name                        12000 non-null  object        
 3   email                       12000 non-null  object        
 4   creation_source             12000 non-null  object        
 5   last_session_creation_time  8823 non-null   float64       
 6   opted_in_to_mailing_list    12000 non-null  int64         
 7   enabled_for_marketing_drip  12000 non-null  int64         
 8   org_id                      12000 non-null  int64         
 9   invited_by_user_id          6417 non-null   float64       
dtypes: datetime64[ns](1), float64(2), int64(4), object(3)
memory usage: 937.6+ KB


In [4]:
# load engagement data
engagement = pd.read_csv('takehome_user_engagement.csv',parse_dates=['time_stamp'])
engagement.head()

,time_stamp,user_id,visited
0,2014-04-22 03:53:30,1,1
1,2013-11-15 03:45:04,2,1
2,2013-11-29 03:45:04,2,1
3,2013-12-09 03:45:04,2,1
4,2013-12-25 03:45:04,2,1


In [5]:
engagement.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 207917 entries, 0 to 207916
Data columns (total 3 columns):
 #   Column      Non-Null Count   Dtype         
---  ------      --------------   -----         
 0   time_stamp  207917 non-null  datetime64[ns]
 1   user_id     207917 non-null  int64         
 2   visited     207917 non-null  int64         
dtypes: datetime64[ns](1), int64(2)
memory usage: 4.8 MB


**Define Adopted User**

In [6]:
# create multi index, group by and do 7 day rolling sum of visits for engagement table 
engagement.set_index(['user_id','time_stamp'], inplace=True)
engagement['7day_visits'] = engagement.groupby(level='user_id').apply(lambda x: x.rolling(window=7).sum())

In [7]:
# filter only those with visits >=3 in window
adopted = engagement[engagement['7day_visits'] >= 3]
adopted.head()

visited  7day_visits
user_id time_stamp                               
2       2014-02-03 03:45:04        1          7.0
        2014-02-08 03:45:04        1          7.0
        2014-02-09 03:45:04        1          7.0
        2014-02-13 03:45:04        1          7.0
        2014-02-16 03:45:04        1          7.0

In [8]:
# melt to get user_id again
adopted = adopted.reset_index().melt(id_vars = ['user_id'], value_vars = ['7day_visits'])

In [9]:
adopted = adopted['user_id'].unique()
len(adopted)

1631

#### Add Adopted User Flag To Users Table

In [10]:
users['adopted'] = users['object_id'].isin(adopted)
users['adopted'] = users['adopted'].apply(lambda x: 1 if x == True else 0)

#### Additional Cleaning

In [11]:
# New columns for day, week month,year
users['creation_year'] = users.creation_time.dt.year
users['creation_month'] = users.creation_time.dt.month
users['creation_week'] = users.creation_time.dt.week
users['creation_day'] = users.creation_time.dt.day

In [12]:
users.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12000 entries, 0 to 11999
Data columns (total 15 columns):
 #   Column                      Non-Null Count  Dtype         
---  ------                      --------------  -----         
 0   object_id                   12000 non-null  int64         
 1   creation_time               12000 non-null  datetime64[ns]
 2   name                        12000 non-null  object        
 3   email                       12000 non-null  object        
 4   creation_source             12000 non-null  object        
 5   last_session_creation_time  8823 non-null   float64       
 6   opted_in_to_mailing_list    12000 non-null  int64         
 7   enabled_for_marketing_drip  12000 non-null  int64         
 8   org_id                      12000 non-null  int64         
 9   invited_by_user_id          6417 non-null   float64       
 10  adopted                     12000 non-null  int64         
 11  creation_year               12000 non-null  int64     

In [13]:
# drop columns not needed
cols_drop = ['last_session_creation_time','name','email']
users.drop(cols_drop, axis=1,inplace=True)

In [14]:
# fill missing values with '0', to indicate no user recommended them 
users['invited_by_user_id'] = users['invited_by_user_id'].fillna(0)

# add new column with flag of whether or not the user was invited by another user 
users['invited_by_user'] = users['invited_by_user_id'].apply(lambda x: 1 if x > 0 else 0)

users.head()

,object_id,creation_time,creation_source,opted_in_to_mailing_list,enabled_for_marketing_drip,org_id,invited_by_user_id,adopted,creation_year,creation_month,creation_week,creation_day,invited_by_user
0,1,2014-04-22 03:53:30,GUEST_INVITE,1,0,11,10803.0,0,2014,4,17,22,1
1,2,2013-11-15 03:45:04,ORG_INVITE,0,0,1,316.0,1,2013,11,46,15,1
2,3,2013-03-19 23:14:52,ORG_INVITE,0,0,94,1525.0,0,2013,3,12,19,1
3,4,2013-05-21 08:09:28,GUEST_INVITE,0,0,1,5151.0,0,2013,5,21,21,1
4,5,2013-01-17 10:14:20,GUEST_INVITE,0,0,193,5240.0,0,2013,1,3,17,1


In [15]:
users.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12000 entries, 0 to 11999
Data columns (total 13 columns):
 #   Column                      Non-Null Count  Dtype         
---  ------                      --------------  -----         
 0   object_id                   12000 non-null  int64         
 1   creation_time               12000 non-null  datetime64[ns]
 2   creation_source             12000 non-null  object        
 3   opted_in_to_mailing_list    12000 non-null  int64         
 4   enabled_for_marketing_drip  12000 non-null  int64         
 5   org_id                      12000 non-null  int64         
 6   invited_by_user_id          12000 non-null  float64       
 7   adopted                     12000 non-null  int64         
 8   creation_year               12000 non-null  int64         
 9   creation_month              12000 non-null  int64         
 10  creation_week               12000 non-null  int64         
 11  creation_day                12000 non-null  int64     

In [16]:
users.describe()

,object_id,opted_in_to_mailing_list,enabled_for_marketing_drip,org_id,invited_by_user_id,adopted,creation_year,creation_month,creation_week,creation_day,invited_by_user
count,12000.00000,12000.000000,12000.000000,12000.000000,12000.000000,12000.000000,12000.000000,12000.000000,12000.000000,12000.000000,12000.000000
mean,6000.50000,0.249500,0.149333,141.884583,3188.691333,0.135917,2013.060833,6.250833,25.360000,15.986583,0.534750
std,3464.24595,0.432742,0.356432,124.056723,3869.027693,0.342714,0.723424,3.423968,14.871343,8.803533,0.498812
min,1.00000,0.000000,0.000000,0.000000,0.000000,0.000000,2012.000000,1.000000,1.000000,1.000000,0.000000
25%,3000.75000,0.000000,0.000000,29.000000,0.000000,0.000000,2013.000000,3.000000,13.000000,8.000000,0.000000
50%,6000.50000,0.000000,0.000000,108.000000,875.000000,0.000000,2013.000000,6.000000,23.000000,16.000000,1.000000
75%,9000.25000,0.000000,0.000000,238.250000,6317.000000,0.000000,2014.000000,9.000000,38.000000,24.000000,1.000000
max,12000.00000,1.000000,1.000000,416.000000,11999.000000,1.000000,2014.000000,12.000000,52.000000,31.000000,1.000000


#### Export Cleaned File

In [17]:
users.to_csv('takehome_users_cleaned.csv')